## Dataset 1: MovieLens 100K (Explicit Ratings)

### 🔹 Setup Summary
- **Data type:** Explicit ratings (1–5 stars)
- **Filtering:** 
    - users ≥ 20 ratings
    - movies ≥ 5 ratings
- **Data Splitting:** Training/Test split using u2.base (train) & u2.test (test), 80/20 proportion by timestamp or random selection
- **Model:** Matrix Factorization (MF) for Collaborative Filtering
- **Optimization Methods Tested:** Stochastic Gradient Descent (SGD)
- **Parameters Tested:** 
    - latent factors = 20, 50, 100
    - regularization λ = 0.01, 0.02, 0.05
    - learning rate (for SGD) = 0.001, 0.002, 0.005
    - iterations = 50, 100
- **Evaluation Metrics:** RMSE, MAE (for prediction accuracy), Precision@5, Recall@5, Precision@10, Recall@10, NDCG@10

In [4]:
# Load MovieLens 100K dataset
import pandas as pd

train = pd.read_csv('data/ml-100k/u2.base', sep='\t',
                    names=['user_id','item_id','rating','timestamp'])
test  = pd.read_csv('data/ml-100k/u2.test', sep='\t',
                    names=['user_id','item_id','rating','timestamp'])

In [5]:
# --- Filtering ---
# Keep users with at least 20 ratings
user_counts = train['user_id'].value_counts()
train = train[train['user_id'].isin(user_counts[user_counts >= 20].index)]

# Keep items with at least 5 ratings
item_counts = train['item_id'].value_counts()
train = train[train['item_id'].isin(item_counts[item_counts >= 5].index)]

# Ensure test set only contains filtered users/items
test = test[test['user_id'].isin(train['user_id'].unique())]
test = test[test['item_id'].isin(train['item_id'].unique())]

In [6]:
# Print dataset shapes
print(f"Train set shape: {train.shape}")
print(f"Test set shape: {test.shape}\n")

# Preview first few rows
print("Train set preview:")
display(train.head())

print("\nTest set preview:")
display(test.head())

Train set shape: (77487, 4)
Test set shape: (18892, 4)

Train set preview:


,user_id,item_id,rating,timestamp
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561



Test set preview:


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,8,1,875072484
3,1,9,5,878543541
4,1,21,1,878542772


In [7]:
# Rating statistics
rating_stats = train['rating'].describe()

print("Rating Statistics:")
print(rating_stats[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']])

Rating Statistics:
count    77487.000000
mean         3.533483
std          1.118780
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: rating, dtype: float64


In [8]:
# Display dataset information
info_path = 'data/ml-100k/u.info'

with open(info_path, 'r') as f:
    info = f.read()

print("MovieLens 100K Dataset Info:\n")
print(info)

MovieLens 100K Dataset Info:

943 users
1682 items
100000 ratings



In [9]:
# Load MovieLens movie metadata
import pandas as pd

column_names = [
    'movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL',
    'unknown', 'Action', 'Adventure', 'Animation', "Children's",
    'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
    'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
    'War', 'Western'
]

movie_path = 'data/ml-100k/u.item'
u_item = pd.read_csv(movie_path, sep='|', names=column_names, encoding='latin-1')

# Display the first few rows
u_item.head()

,movie_id,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
import numpy as np
import pandas as pd

# Create User-Item Matrix
n_users = train['user_id'].max()
n_items = train['item_id'].max()

def build_user_item_matrix(df, n_users, n_items):
    """Builds a user-item rating matrix from a DataFrame."""
    R = np.zeros((n_users, n_items))
    for row in df.itertuples():
        R[row.user_id - 1, row.item_id - 1] = row.rating
    return R

# Build matrices for training and testing
R_train = build_user_item_matrix(train, n_users, n_items)
R_test = build_user_item_matrix(test, n_users, n_items)

# Display basic info
print(f"R_train shape: {R_train.shape}, nonzero entries: {np.count_nonzero(R_train)}")
print(f"R_test shape: {R_test.shape}, nonzero entries: {np.count_nonzero(R_test)}")

# Preview small subset (first 5 users × first 10 items)
print("\nR_train preview:")
display(pd.DataFrame(R_train[:5, :10]))

print("\nR_test preview:")
display(pd.DataFrame(R_test[:5, :10]))

R_train shape: (943, 1620), nonzero entries: 77487
R_test shape: (943, 1620), nonzero entries: 18892

R_train preview:


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,4.0,3.0,3.0,5.0,4.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



R_test preview:


,0,1,2,3,4,5,6,7,8,9
0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Matrix Factorization Function (SGD)
def matrix_factorization(R, K=20, steps=100, alpha=0.002, beta=0.02, min_rating=1, max_rating=5):
    """
    Perform matrix factorization using Stochastic Gradient Descent (SGD).

    Parameters:
        R (ndarray): User-item rating matrix
        K (int): Number of latent factors
        steps (int): Number of iterations
        alpha (float): Learning rate
        beta (float): Regularization parameter
        min_rating (float): Minimum allowed rating
        max_rating (float): Maximum allowed rating

    Returns:
        ndarray: Predicted rating matrix
    """
    N, M = R.shape

    # Initialize user and item latent factor matrices
    P = np.random.rand(N, K)
    Q = np.random.rand(M, K).T  # Transpose for easier multiplication

    # SGD optimization
    for step in range(steps):
        for i in range(N):
            for j in range(M):
                if R[i, j] > 0:  # Only consider observed ratings
                    eij = R[i, j] - np.dot(P[i, :], Q[:, j])
                    P[i, :] += alpha * (2 * eij * Q[:, j] - beta * P[i, :])
                    Q[:, j] += alpha * (2 * eij * P[i, :] - beta * Q[:, j])

    # Compute final predicted rating matrix and clip to valid range
    return np.dot(P, Q).clip(min_rating, max_rating)


In [12]:
# Evaluation Metrics for Recommendation
import numpy as np

def precision_at_k(recommended, relevant, k):
    """
    Compute Precision@K.
    
    Parameters:
        recommended (list): List of recommended item IDs
        relevant (list or set): List or set of relevant item IDs
        k (int): Number of top recommendations to consider
        
    Returns:
        float: Precision@K
    """
    recommended_k = recommended[:k]
    return len(set(recommended_k) & set(relevant)) / k

def recall_at_k(recommended, relevant, k):
    """
    Compute Recall@K.
    
    Parameters:
        recommended (list): List of recommended item IDs
        relevant (list or set): List or set of relevant item IDs
        k (int): Number of top recommendations to consider
        
    Returns:
        float: Recall@K
    """
    relevant = set(relevant)
    if len(relevant) == 0:
        return 0.0
    recommended_k = recommended[:k]
    return len(set(recommended_k) & relevant) / len(relevant)

def ndcg_at_k(recommended, relevant, k):
    """
    Compute Normalized Discounted Cumulative Gain (NDCG)@K.
    
    Parameters:
        recommended (list): List of recommended item IDs
        relevant (list or set): List or set of relevant item IDs
        k (int): Number of top recommendations to consider
        
    Returns:
        float: NDCG@K
    """
    recommended_k = recommended[:k]
    dcg = sum(1 / np.log2(i + 2) for i, item in enumerate(recommended_k) if item in relevant)
    # Ideal DCG (best possible ranking)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant), k)))
    return dcg / idcg if idcg > 0 else 0

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Main Experiment Setup
K_values = [20, 50, 100]
alpha_values = [0.001, 0.002, 0.005]
beta_values = [0.01, 0.02, 0.05]
step_values = [50, 100]
k_list = [5, 10]

results = []

hyperparams = [
    (K, alpha, beta, steps)
    for K in K_values
    for alpha in alpha_values
    for beta in beta_values
    for steps in step_values
]

for K, alpha, beta, steps in tqdm(hyperparams, desc="Hyperparameter Search", ncols=120):
    # --- Train the model ---
    R_pred = matrix_factorization(R_train, K=K, steps=steps, alpha=alpha, beta=beta)

    # --- Initialize metric storage ---
    metrics = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in k_list}
    true_vals = []
    pred_vals = []

    # --- Evaluate for each user ---
    for user in tqdm(range(R_test.shape[0]), desc="Evaluating users", leave=False, dynamic_ncols=True):
        train_items = set(np.where(R_train[user, :] > 0)[0])
        test_items = set(np.where(R_test[user, :] > 0)[0])
        if not test_items:
            continue

        # Collect true and predicted values for RMSE/MAE
        for i in test_items:
            true_vals.append(R_test[user, i])
            pred_vals.append(R_pred[user, i])
            
        # Get ranking of items by predicted score (excluding already rated items)
        ranked_items = [i for i in np.argsort(R_pred[user, :])[::-1] if i not in train_items]

        # Compute metrics for each K
        for Kval in k_list:
            metrics[Kval]['precision'].append(precision_at_k(ranked_items, test_items, Kval))
            metrics[Kval]['recall'].append(recall_at_k(ranked_items, test_items, Kval))
            metrics[Kval]['ndcg'].append(ndcg_at_k(ranked_items, test_items, Kval))

        # --- Compute RMSE and MAE ---
        rmse = np.sqrt(mean_squared_error(true_vals, pred_vals))
        mae = mean_absolute_error(true_vals, pred_vals)

    # --- Aggregate results ---
    results.append({
        'K': K, 'alpha': alpha, 'beta': beta, 'steps': steps,
        'rmse': rmse,
        'mae': mae,
        **{f'precision@{k}': np.mean(metrics[k]['precision']) for k in k_list},
        **{f'recall@{k}': np.mean(metrics[k]['recall']) for k in k_list},
        **{f'ndcg@{k}': np.mean(metrics[k]['ndcg']) for k in k_list},
    })


# --- Display top results sorted by NDCG@10 ---
results_df = pd.DataFrame(results).sort_values('ndcg@10', ascending=False)
display(results_df.head(10))

Training MF: K=20, α=0.001, β=0.01, steps=50


Evaluating users:   0%|          | 0/943 [00:00<?, ?it/s]

Training MF: K=20, α=0.001, β=0.01, steps=100


Evaluating users:   0%|          | 0/943 [00:00<?, ?it/s]

Training MF: K=20, α=0.001, β=0.02, steps=50


...

Training MF: K=100, α=0.005, β=0.05, steps=50


Evaluating users:   0%|          | 0/943 [00:00<?, ?it/s]

Training MF: K=100, α=0.005, β=0.05, steps=100


Evaluating users:   0%|          | 0/943 [00:00<?, ?it/s]

,K,alpha,beta,steps,rmse,mae,precision@5,precision@10,recall@5,recall@10,ndcg@5,ndcg@10
12,20,0.005,0.01,50,1.077592,0.827063,0.055780,0.052294,0.007946,0.015293,0.053938,0.052832
13,20,0.005,0.01,100,1.126445,0.853988,0.050642,0.051009,0.006178,0.014648,0.052709,0.052605
14,20,0.005,0.02,50,1.057242,0.810976,0.042936,0.049358,0.005136,0.012424,0.042539,0.047462
15,20,0.005,0.02,100,1.097651,0.837336,0.035229,0.035596,0.004887,0.009958,0.037251,0.037062
7,20,0.002,0.01,100,1.050594,0.805462,0.029725,0.033028,0.004815,0.009857,0.031269,0.033333
9,20,0.002,0.02,100,1.036174,0.797176,0.029358,0.029908,0.003591,0.007624,0.030600,0.030420
8,20,0.002,0.02,50,1.002354,0.776929,0.027156,0.026422,0.004231,0.007927,0.028546,0.027720
6,20,0.002,0.01,50,1.012798,0.787951,0.026055,0.026606,0.003725,0.006784,0.027523,0.027604
16,20,0.005,0.05,50,1.014708,0.782444,0.026055,0.026239,0.004376,0.008221,0.025383,0.025994
1,20,0.001,0.01,100,1.006830,0.777819,0.024587,0.024587,0.003302,0.007260,0.025681,0.025529


In [14]:
# --- Train final Matrix Factorization model using best hyperparameters ---
best_row = results_df.iloc[0]
best_params = {
    'K': int(best_row['K']),
    'steps': int(best_row['steps']),
    'alpha': float(best_row['alpha']),
    'beta': float(best_row['beta'])
}
R_pred = matrix_factorization(
    R_train,
    K=best_params['K'],
    steps=best_params['steps'],
    alpha=best_params['alpha'],
    beta=best_params['beta']
)
print("Final MF model trained with best hyperparameters:", best_params)

Final MF model trained with best hyperparameters: {'K': 20, 'steps': 50, 'alpha': 0.005, 'beta': 0.01}


In [18]:
# --- Recommend Top-N items for a single user (return names) ---
def recommend_for_user(R_pred, user_index, train_df, movies_df, N=10):
    """Return top-N recommended movie names for a given user."""
    rated_items = set(train_df[train_df['user_id'] == user_index + 1]['item_id'].values)
    scores = R_pred[user_index]

    unrated_indices = [i for i in range(len(scores)) if (i + 1) not in rated_items]
    top_indices = sorted(unrated_indices, key=lambda x: scores[x], reverse=True)[:N]

    top_item_ids = [i + 1 for i in top_indices]  # convert to 1-indexed IDs
    top_names = movies_df[movies_df['movie_id'].isin(top_item_ids)]['title'].tolist()
    return top_names

# --- Evaluate recommendation metrics (still uses IDs for calculation) ---
def evaluate_user(R_pred, user_index, train_df, test_df, k_list=[5, 10]):
    relevant_items = test_df[test_df['user_id'] == user_index + 1]['item_id'].values
    recommended_items = recommend_for_user_ids(R_pred, user_index, train_df, N=max(k_list))  # return IDs for metrics

    results = {}
    for k in k_list:
        results[f'precision@{k}'] = precision_at_k(recommended_items, relevant_items, k)
        results[f'recall@{k}'] = recall_at_k(recommended_items, relevant_items, k)
        results[f'ndcg@{k}'] = ndcg_at_k(recommended_items, relevant_items, k)
    return results

# Helper function to return IDs for evaluation
def recommend_for_user_ids(R_pred, user_index, train_df, N=10):
    rated_items = set(train_df[train_df['user_id'] == user_index + 1]['item_id'].values)
    scores = R_pred[user_index]

    unrated_indices = [i for i in range(len(scores)) if (i + 1) not in rated_items]
    top_indices = sorted(unrated_indices, key=lambda x: scores[x], reverse=True)[:N]
    return [i + 1 for i in top_indices]

# --- Example Usage ---
user_index = 0  # corresponds to user_id=1
top_movie_names = recommend_for_user(R_pred, user_index, train, u_item, N=10)
metrics = evaluate_user(R_pred, user_index, train, test, k_list=[5, 10])

print(f"\n=== Top 10 Recommended Movies for User {user_index + 1} ===")
for rank, movie in enumerate(top_movie_names, 1):
    print(f"{rank}. {movie}")

print(f"\n--- Evaluation Metrics for User {user_index + 1} ---")
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")



=== Top 10 Recommended Movies for User 1 ===
1. Mr. Smith Goes to Washington (1939)
2. Men in Black (1997)
3. Leaving Las Vegas (1995)
4. Mrs. Brown (Her Majesty, Mrs. Brown) (1997)
5. In the Name of the Father (1993)
6. Close Shave, A (1995)
7. Birds, The (1963)
8. Short Cuts (1993)
9. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
10. Great Escape, The (1963)

--- Evaluation Metrics for User 1 ---
precision@5: 0.4000
recall@5: 0.0392
ndcg@5: 0.5531
precision@10: 0.2000
recall@10: 0.0392
ndcg@10: 0.3590
